In [1]:
pip install fastmcp google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain-google-community[gmail] langchain langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
from mcp.server.fastmcp import FastMCP
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os, base64, email.mime.text

# Scopes for sending mail
SCOPES = ["https://www.googleapis.com/auth/gmail.send"]

class GmailMCPServer:
    def __init__(self, creds_file="credentials.json", token_file="token.json"):
        self.creds_file = creds_file
        self.token_file = token_file
        self.service = self._authenticate_gmail()
        self.mcp = FastMCP("email")

        # Register MCP tools
        @self.mcp.tool()
        def send_email(to: str, subject: str, body: str) -> str:
            """Send an email via Gmail"""
            msg = email.mime.text.MIMEText(body)
            msg["to"] = to
            msg["subject"] = subject
            raw = base64.urlsafe_b64encode(msg.as_bytes()).decode()
            try:
                sent = self.service.users().messages().send(
                    userId="me", body={"raw": raw}
                ).execute()
                return f"Message sent, ID: {sent['id']}"
            except HttpError as e:
                return f"Error: {e}"

    def _authenticate_gmail(self):
        creds = None
        if os.path.exists(self.token_file):
            creds = Credentials.from_authorized_user_file(self.token_file, SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.creds_file, SCOPES)
                creds = flow.run_local_server(port=0)
            with open(self.token_file, "w") as f:
                f.write(creds.to_json())
        return build("gmail", "v1", credentials=creds)

    def run(self):
        self.mcp.run(transport="stdio")

if __name__ == "__main__":
    GmailMCPServer().run()


RefreshError: ('invalid_scope: Bad Request', {'error': 'invalid_scope', 'error_description': 'Bad Request'})